In [1]:
from imports import*

from mf_sim import *
from mf_fit import *
from mf_predict import *

from mb_sim import *
from mb_fit import *
from mb_predict import *

from habit_sim import *
from habit_fit import *
from habit_predict import *

from wsls_sim import *
from wsls_fit import *
from wsls_predict import *

from kdh_sim import *
from kdh_fit import *
from kdh_predict import *

from utils import *
from plot_stats import *
from logistic_regression import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn

In [2]:
#config num of agent to simulate
num_of_agents = 100

#config num of trails for each block
num_of_trials = 200

num_of_block = 5
# for cross valdation 
array = np.arange(num_of_block)
cv = [np.roll(array,i) for i in range(num_of_block)]
cv = np.array(cv)

reversal = 50

# create transtion probs
transtion_probs = np.array([[.8,.2],
                            [.2,.8]])

models = {
        'mf':[configuration_parameters_mf, mf_sim],
        'mb':[configuration_parameters_mb, mb_sim],
        'habit':[configuration_parameters_habits,habit_sim],
        'wsls':[configuration_parameters_wsls,wsls_sim],
        'kdh':[configuration_parameters_kdh,kdh_sim]
         }

In [3]:
# sim
for m in models:
    print(f'Model {m}')
    
    data_per_agent = []
    parameters = []

    for agent in tqdm(range(num_of_agents)):
        param = models[m][0]()
        parameters.append(param)
        
        data = []
        for i in range(num_of_block):
            # create rewards probs 
            reward_probs = create_reward_probs(num_of_trials,reversal,0.2,0.8)
            df = models[m][1](
                            param,
                            num_of_trials,
                            transtion_probs,
                            reward_probs
            ) 
            data.append(df)
        data_per_agent.append(data)

    df = pd.DataFrame(parameters)
    df.to_csv(f'../results/{m}/{m}_parameters.csv')
    
    for agent in range(num_of_agents):
        for block in range(num_of_block):
            data_per_agent[agent][block].to_csv(f'../data/{m}/{m}_agent_{agent}_sim_{block}.csv')
    

Model mf


100%|██████████████████████████████████████████| 100/100 [00:09<00:00, 10.47it/s]


Model mb


100%|██████████████████████████████████████████| 100/100 [00:08<00:00, 11.22it/s]


Model habit


100%|██████████████████████████████████████████| 100/100 [00:08<00:00, 11.21it/s]


Model wsls


100%|██████████████████████████████████████████| 100/100 [00:07<00:00, 12.85it/s]


Model kdh


100%|██████████████████████████████████████████| 100/100 [00:07<00:00, 13.20it/s]


In [5]:
criterion = nn.BCELoss()

for m in tqdm(models):    
    # load data 
    data_per_agent = []
    for agent in range(num_of_agents):
        data = []
        for sim in range(num_of_block):
            data.append(pd.read_csv(f'../data/{m}/{m}_agent_{agent}_sim_{sim}.csv'))
        data_per_agent.append(data)
        
    data_results = {
                   'agent': [], 
                   'fit_parameters_mf': [], 
                   'train_nlp_mf' : [], 
                   'test_acc_mf': [],
                   'test_nlp_mf': [], 

                   'fit_parameters_mb': [], 
                   'train_nlp_mb' : [], 
                   'test_acc_mb': [],
                   'test_nlp_mb': [],
        
                   'fit_parameters_habit': [], 
                   'train_nlp_habit' : [], 
                   'test_acc_habit': [],
                   'test_nlp_habit': [], 
        
                    'fit_parameters_wsls': [], 
                   'train_nlp_wsls' : [], 
                   'test_acc_wsls': [],
                   'test_nlp_wsls': [], 

                   'fit_parameters_kdh': [], 
                   'train_nlp_kdh' : [], 
                   'test_acc_kdh': [],
                   'test_nlp_kdh': [], 

                   'fit_parameters_logistic_regression': [], 
                   'train_nlp_logistic_regression' : [], 
                   'test_acc_logistic_regression': [],
                   'test_nlp_logistic_regression': [],

            }
    print(f'*** Fit {m} ***')
    for agent in range(num_of_agents):
        print(f'* agent {agent} *')
        for n,t in enumerate(cv):
            data_results['agent'].append(agent)
            train_arr = t[0:-1]
            test_arr = t[-1:]

            # split train and test data
            train_data = [data_per_agent[agent][sim] for sim in train_arr]
            train_data = pd.concat(train_data) 
            train_data.reset_index(inplace=True)
            n_train = len(train_data)

            test_data = [data_per_agent[agent][sim] for sim in test_arr]
            test_data = pd.concat(test_data) 
            test_data.reset_index(inplace=True)
            n_test = len(test_data)
            
            # fit mf 
            res = mf_fit(train_data,2)
            data_results['fit_parameters_mf'].append(res.x)

            # Train log probability
            data_results['train_nlp_mf'].append(res.fun/n_train)

            # Test Accuracy and loss
            accuracy, p_0 = mf_predict(test_data,res.x)

            # Test Accuracy
            data_results['test_acc_mf'].append(accuracy/n_test)

            # Test negative log probability
            running_loss = 0
            for row in test_data.itertuples(index=True, name='Pandas'):
                y_pred = torch.tensor([1-p_0[row.Index]],dtype=torch.float32)
                y_true = torch.tensor([row.action_stage_1],dtype=torch.float32)
                running_loss += (criterion(y_pred,y_true)).numpy()
            data_results['test_nlp_mf'].append((running_loss/n_test))
            
            # fit mb
            res = mb_fit(train_data,2)
            data_results['fit_parameters_mb'].append(res.x)

            # Train negative log probability
            data_results['train_nlp_mb'].append(res.fun/n_train)

            # Test Accuracy and loss
            accuracy, p_0 = mb_predict(test_data,res.x)

            # Test Accuracy
            data_results['test_acc_mb'].append(accuracy/n_test)

            # Test negative log probability
            running_loss = 0
            for row in test_data.itertuples(index=True, name='Pandas'):
                y_pred = torch.tensor([1-p_0[row.Index]],dtype=torch.float32)
                y_true = torch.tensor([row.action_stage_1],dtype=torch.float32)
                running_loss += (criterion(y_pred,y_true)).numpy()
            data_results['test_nlp_mb'].append((running_loss/n_test))
            
            # fit habit
            res = habit_fit(train_data,2)
            data_results['fit_parameters_habit'].append(res.x)

            # Train negative log probability
            data_results['train_nlp_habit'].append(res.fun/n_train)

            # Test Accuracy and loss
            accuracy, p_0 = habit_predict(test_data,res.x)

            # Test Accuracy
            data_results['test_acc_habit'].append(accuracy/n_test)

            # Test negative log probability
            running_loss = 0
            for row in test_data.itertuples(index=True, name='Pandas'):
                y_pred = torch.tensor([1-p_0[row.Index]],dtype=torch.float32)
                y_true = torch.tensor([row.action_stage_1],dtype=torch.float32)
                running_loss += (criterion(y_pred,y_true)).numpy()
            data_results['test_nlp_habit'].append((running_loss/n_test))
            
            # fit wsls
            res = wsls_fit(train_data,2)
            data_results['fit_parameters_wsls'].append(res.x)

            # Train negative log probability
            data_results['train_nlp_wsls'].append(res.fun/n_train)

            # Test Accuracy and loss
            accuracy, p_0 = wsls_predict(test_data,res.x)

            # Test Accuracy
            data_results['test_acc_wsls'].append(accuracy/n_test)

            # Test negative log probability
            running_loss = 0
            for row in test_data.itertuples(index=True, name='Pandas'):
                y_pred = torch.tensor([1-p_0[row.Index]],dtype=torch.float32)
                y_true = torch.tensor([row.action_stage_1],dtype=torch.float32)
                running_loss += (criterion(y_pred,y_true)).numpy()
            data_results['test_nlp_wsls'].append((running_loss/n_test))
            
            # fit kDH
            res = kdh_fit(train_data,2)
            data_results['fit_parameters_kdh'].append(res.x)

            # Train negative log probability
            data_results['train_nlp_kdh'].append(res.fun/n_train)

            # Test Accuracy and loss
            accuracy, p_0 = kdh_predict(test_data,res.x)

            # Test Accuracy
            data_results['test_acc_kdh'].append(accuracy/n_test)

            # Test negative log probability
            running_loss = 0
            for row in test_data.itertuples(index=True, name='Pandas'):
                y_pred = torch.tensor([1-p_0[row.Index]],dtype=torch.float32)
                y_true = torch.tensor([row.action_stage_1],dtype=torch.float32)
                running_loss += (criterion(y_pred,y_true)).numpy()
            data_results['test_nlp_kdh'].append((running_loss/n_test))

            X , y = preprocess_logistic_regression(train_data)
            model, intercept, coef  = fit_logistic_regression(X,y)
            data_results['fit_parameters_logistic_regression'].append([intercept,coef])

            # Train negative log probability
            nlp = nlp_logistic_regression(model,X,y)
            data_results['train_nlp_logistic_regression'].append((nlp/n_train))

            # test data 
            X, y = preprocess_logistic_regression(test_data)
            # Test Accuracy
            accuracy = model.score(X,y)
            data_results['test_acc_logistic_regression'].append(accuracy)

            # Test negative log probability
            nlp = nlp_logistic_regression(model,X,y)
            data_results['test_nlp_logistic_regression'].append((nlp/n_test))


            
    print(f'** save results {m} **')
    # save data 
    df = pd.DataFrame(data_results)
    df.to_csv(f'../results/{m}/{m}_fit.csv')

  0%|                                                      | 0/5 [00:00<?, ?it/s]

*** Fit mf ***
* agent 0 *
* agent 1 *
* agent 2 *
* agent 3 *
* agent 4 *
* agent 5 *
* agent 6 *
* agent 7 *
* agent 8 *
* agent 9 *
* agent 10 *
* agent 11 *
* agent 12 *
* agent 13 *
* agent 14 *
* agent 15 *
* agent 16 *
* agent 17 *
* agent 18 *
* agent 19 *
* agent 20 *
* agent 21 *
* agent 22 *
* agent 23 *
* agent 24 *
* agent 25 *
* agent 26 *
* agent 27 *
* agent 28 *
* agent 29 *
* agent 30 *
* agent 31 *
* agent 32 *
* agent 33 *
* agent 34 *
* agent 35 *
* agent 36 *
* agent 37 *
* agent 38 *
* agent 39 *
* agent 40 *
* agent 41 *
* agent 42 *
* agent 43 *
* agent 44 *
* agent 45 *
* agent 46 *
* agent 47 *
* agent 48 *
* agent 49 *
* agent 50 *
* agent 51 *
* agent 52 *
* agent 53 *
* agent 54 *
* agent 55 *
* agent 56 *
* agent 57 *
* agent 58 *
* agent 59 *
* agent 60 *
* agent 61 *
* agent 62 *
* agent 63 *
* agent 64 *
* agent 65 *
* agent 66 *
* agent 67 *
* agent 68 *
* agent 69 *
* agent 70 *
* agent 71 *
* agent 72 *
* agent 73 *
* agent 74 *
* agent 75 *
* agent

 20%|█████████                                    | 1/5 [08:23<33:35, 503.90s/it]

*** Fit mb ***
* agent 0 *
* agent 1 *
* agent 2 *
* agent 3 *
* agent 4 *
* agent 5 *
* agent 6 *
* agent 7 *
* agent 8 *
* agent 9 *
* agent 10 *
* agent 11 *
* agent 12 *
* agent 13 *
* agent 14 *
* agent 15 *
* agent 16 *
* agent 17 *
* agent 18 *
* agent 19 *
* agent 20 *
* agent 21 *
* agent 22 *
* agent 23 *
* agent 24 *
* agent 25 *
* agent 26 *
* agent 27 *
* agent 28 *
* agent 29 *
* agent 30 *
* agent 31 *
* agent 32 *
* agent 33 *
* agent 34 *
* agent 35 *
* agent 36 *
* agent 37 *
* agent 38 *
* agent 39 *
* agent 40 *
* agent 41 *
* agent 42 *
* agent 43 *
* agent 44 *
* agent 45 *
* agent 46 *
* agent 47 *
* agent 48 *
* agent 49 *
* agent 50 *
* agent 51 *
* agent 52 *
* agent 53 *
* agent 54 *
* agent 55 *
* agent 56 *
* agent 57 *
* agent 58 *
* agent 59 *
* agent 60 *
* agent 61 *
* agent 62 *
* agent 63 *
* agent 64 *
* agent 65 *
* agent 66 *
* agent 67 *
* agent 68 *
* agent 69 *
* agent 70 *
* agent 71 *
* agent 72 *
* agent 73 *
* agent 74 *
* agent 75 *
* agent

 40%|██████████████████                           | 2/5 [16:32<24:44, 494.97s/it]

...

 60%|███████████████████████████                  | 3/5 [24:27<16:11, 485.90s/it]

*** Fit wsls ***
* agent 0 *
* agent 1 *
* agent 2 *
* agent 3 *
* agent 4 *
* agent 5 *
* agent 6 *
* agent 7 *
* agent 8 *
* agent 9 *
* agent 10 *
* agent 11 *
* agent 12 *
* agent 13 *
* agent 14 *
* agent 15 *
* agent 16 *
* agent 17 *
* agent 18 *
* agent 19 *
* agent 20 *
* agent 21 *
* agent 22 *
* agent 23 *
* agent 24 *
* agent 25 *
* agent 26 *
* agent 27 *
* agent 28 *
* agent 29 *
* agent 30 *
* agent 31 *
* agent 32 *
* agent 33 *
* agent 34 *
* agent 35 *
* agent 36 *
* agent 37 *
* agent 38 *
* agent 39 *
* agent 40 *
* agent 41 *
* agent 42 *
* agent 43 *
* agent 44 *
* agent 45 *
* agent 46 *
* agent 47 *
* agent 48 *
* agent 49 *
* agent 50 *
* agent 51 *
* agent 52 *
* agent 53 *
* agent 54 *
* agent 55 *
* agent 56 *
* agent 57 *
* agent 58 *
* agent 59 *
* agent 60 *
* agent 61 *
* agent 62 *
* agent 63 *
* agent 64 *
* agent 65 *
* agent 66 *
* agent 67 *
* agent 68 *
* agent 69 *
* agent 70 *
* agent 71 *
* agent 72 *
* agent 73 *
* agent 74 *
* agent 75 *
* age

 80%|████████████████████████████████████         | 4/5 [30:27<07:16, 436.21s/it]

*** Fit kdh ***
* agent 0 *
* agent 1 *
* agent 2 *
* agent 3 *
* agent 4 *
* agent 5 *
* agent 6 *
* agent 7 *
* agent 8 *
* agent 9 *
* agent 10 *
* agent 11 *
* agent 12 *
* agent 13 *
* agent 14 *
* agent 15 *
* agent 16 *
* agent 17 *
* agent 18 *
* agent 19 *
* agent 20 *
* agent 21 *
* agent 22 *
* agent 23 *
* agent 24 *
* agent 25 *
* agent 26 *
* agent 27 *
* agent 28 *
* agent 29 *
* agent 30 *
* agent 31 *
* agent 32 *
* agent 33 *
* agent 34 *
* agent 35 *
* agent 36 *
* agent 37 *
* agent 38 *
* agent 39 *
* agent 40 *
* agent 41 *
* agent 42 *
* agent 43 *
* agent 44 *
* agent 45 *
* agent 46 *
* agent 47 *
* agent 48 *
* agent 49 *
* agent 50 *
* agent 51 *
* agent 52 *
* agent 53 *
* agent 54 *
* agent 55 *
* agent 56 *
* agent 57 *
* agent 58 *
* agent 59 *
* agent 60 *
* agent 61 *
* agent 62 *
* agent 63 *
* agent 64 *
* agent 65 *
* agent 66 *
* agent 67 *
* agent 68 *
* agent 69 *
* agent 70 *
* agent 71 *
* agent 72 *
* agent 73 *
* agent 74 *
* agent 75 *
* agen

100%|█████████████████████████████████████████████| 5/5 [42:55<00:00, 515.16s/it]


In [6]:
Model = ['mf','mb','habit','wsls','kdh']
all_df = []
for m in Model:
    df = pd.read_csv(f'../results/{m}/{m}_fit.csv')
    col = [name for name in df.columns if 'test_nlp' in name]
    df = pd.DataFrame(df[col].describe().loc['mean'])
    df.rename(columns={'mean':f'true_{m}'},inplace=True) 
    df = df.round(3)
    all_df.append(df)
    
d = pd.concat(all_df, axis=1)

d.rename(index={'test_nlp_mf':'fit_mf',
                'test_nlp_mb':'fit_mb',
                'test_nlp_habit':'fit_habit',
                'test_nlp_lwsls':'fit_wsls',
                'test_nlp_kdh':'fit_kdh',
                'test_nlp_logistic_regression':'fit_logistic_regression'},inplace=True)
lr = pd.DataFrame(d.iloc[5])

d.style.background_gradient(cmap ='viridis').set_properties(**{'font-size': '20px'})

,true_mf,true_mb,true_habit,true_wsls,true_kdh
fit_mf,0.396000,0.564000,0.676000,0.661000,0.613000
fit_mb,0.538000,0.460000,0.684000,0.684000,0.655000
fit_habit,0.496000,0.555000,0.457000,0.637000,0.590000
test_nlp_wsls,0.465000,0.571000,0.478000,0.490000,0.622000
fit_kdh,0.701000,0.694000,0.712000,0.695000,0.491000
fit_logistic_regression,0.461000,0.524000,0.476000,0.487000,0.619000
